In [588]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from scipy.stats import t
from scipy.stats import ttest_ind, shapiro, f_oneway, mannwhitneyu
from statsmodels.stats.proportion import proportions_ztest

In [589]:
hotels = pd.read_csv('data/hotels.csv')
hotels.head(3)

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,tags,days_since_review,lat,lng
0,Stratton Street Mayfair Westminster Borough Lo...,581,2/19/2016,8.4,The May Fair Hotel,United Kingdom,Leaving,3,1994,Staff were amazing,4,7,10.0,"[' Leisure trip ', ' Couple ', ' Studio Suite ...",531 day,51.507894,-0.143671
1,130 134 Southampton Row Camden London WC1B 5AF...,299,1/12/2017,8.3,Mercure London Bloomsbury Hotel,United Kingdom,poor breakfast,3,1361,location,2,14,6.3,"[' Business trip ', ' Couple ', ' Standard Dou...",203 day,51.521009,-0.123097
2,151 bis Rue de Rennes 6th arr 75006 Paris France,32,10/18/2016,8.9,Legend Saint Germain by Elegancia,China,No kettle in room,6,406,No Positive,0,14,7.5,"[' Leisure trip ', ' Solo traveler ', ' Modern...",289 day,48.845377,2.325643


## <center> Выполнение заданий в рамках модуля

In [590]:
hotels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386803 entries, 0 to 386802
Data columns (total 17 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   hotel_address                               386803 non-null  object 
 1   additional_number_of_scoring                386803 non-null  int64  
 2   review_date                                 386803 non-null  object 
 3   average_score                               386803 non-null  float64
 4   hotel_name                                  386803 non-null  object 
 5   reviewer_nationality                        386803 non-null  object 
 6   negative_review                             386803 non-null  object 
 7   review_total_negative_word_counts           386803 non-null  int64  
 8   total_number_of_reviews                     386803 non-null  int64  
 9   positive_review                             386803 non-null  object 
 

Количество отелей, представленных в наборе данных

In [591]:
hotels['hotel_name'].nunique()

1492

Перевод даты в формат datetime

In [592]:
hotels['review_date'] = pd.to_datetime(hotels['review_date'])

In [593]:
print(hotels['review_date'].max())
print(hotels['review_date'].min())

2017-08-03 00:00:00
2015-08-04 00:00:00


Количество уникальных тегов

In [594]:
unique_tags = []
for tag_list in hotels['tags']:
    for tag in tag_list[2:-2].split('\', \''):
                
        if tag.strip() not in unique_tags:
            unique_tags.append(tag.strip())
        
        
            

print(f'Число уникальных тегов: {len(unique_tags)}')

Число уникальных тегов: 2368


Какой тег представлен в наибольшем количестве отзывов

In [595]:
tags = []
for tag_list in hotels['tags']:
    for tag in tag_list[2:-2].split('\', \''):
        tags.append(tag.strip())
        
from collections import Counter
cnt = Counter(tags)
cnt.most_common()[0]

('Leisure trip', 313593)

На сколько ночей чаще всего останавливаются (из тегов)

In [596]:
tags = []
for tag_list in hotels['tags']:
    for tag in tag_list[2:-2].split('\', \''):
        tags.append(tag.strip())
        
from collections import Counter
cnt = Counter(tags)
for i in cnt.most_common():
    if 'night' in i[0]:
        print(i[0], i[1])


Stayed 1 night 145373
Stayed 2 nights 100263
Stayed 3 nights 72000
Stayed 4 nights 35748
Stayed 5 nights 15611
Stayed 6 nights 7399
Stayed 7 nights 5549
Stayed 8 nights 1910
Stayed 9 nights 966
Stayed 10 nights 663
Stayed 11 nights 306
Stayed 12 nights 217
Stayed 14 nights 184
Stayed 13 nights 174
Stayed 15 nights 87
Stayed 16 nights 38
Stayed 17 nights 27
Stayed 18 nights 24
Stayed 19 nights 23
Stayed 21 nights 19
Stayed 20 nights 17
Stayed 27 nights 10
Stayed 30 nights 10
Stayed 22 nights 8
King Room with Knightsbridge View 8
Stayed 28 nights 7
Stayed 26 nights 6
Special Offer Double Room 3 nights minimum 6
Stayed 23 nights 6
Stayed 24 nights 5
Junior Suite Give me the night 4
Stayed 25 nights 4
Stayed 29 nights 3
Suite with Knightsbridge View 1
Knightsbridge Suite 1


## <center> Создаем отчет EDA SWEETWIZ

In [597]:
# import sweetviz as sv

# report = sv.analyze(hotels)
# report.show_html()

## <center> 1. Кодирование и Удаление строковых значений. Небольшие преобразования

Необходимо удалить из набора данных столбцы, данные в которых представлены не числами.

Работа с признаком адреса

In [598]:
# функция выделения из текста города
def adress_analysis(text):
    if text.split()[-1] == 'Kingdom':
        city = text.split()[-5]
        country = ' '.join(text.split()[-2:])
    else:
        city = text.split()[-2]
        country = text.split()[-1]
    return city

In [599]:
# создаем признак города
hotels['city'] = hotels['hotel_address'].apply(adress_analysis)

In [600]:
# удаляем признак адреса
hotels = hotels.drop(columns=['hotel_address'])

Кодируем признак названия и удаляем его

In [601]:
# для признака названия отеля производим двоичную кодировку
import category_encoders as ce # импорт для работы с кодировщиком
bin_encoder = ce.BinaryEncoder(cols=['hotel_name']) # указываем столбец для кодирования
type_bin = bin_encoder.fit_transform(hotels['hotel_name'])
hotels = pd.concat([hotels, type_bin], axis=1)

# удаляем признак названия отеля
hotels = hotels.drop(columns=['hotel_name'])

Кодируем признак национальности и удаляем его

In [602]:
# убираем лишние пробелы из элементов признака
hotels['reviewer_nationality'] = hotels['reviewer_nationality'].apply(lambda x: x.strip().lower())
# создаем список из 10 наиболее часто встречающихся национальностей
most_common_nations = hotels['reviewer_nationality'].value_counts(normalize=True)[:10].index.tolist()
# оставляем только национальности из списка, остальные Other
hotels['reviewer_nationality'] = hotels['reviewer_nationality'].apply(lambda x: x if x in most_common_nations else 'other')

# применяем однократное кодирование для признака (11 признаков)
import category_encoders as ce # импорт для работы с кодировщиком

encoder = ce.OneHotEncoder(cols=['reviewer_nationality']) # указываем столбец для кодирования
type_bin = encoder.fit_transform(hotels['reviewer_nationality'])
hotels = pd.concat([hotels, type_bin], axis=1)

# удаляем исходный признак национальности
hotels = hotels.drop(columns=['reviewer_nationality'])

C:\Users\dolzh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\category_encoders\one_hot.py:145: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for cat_name, class_ in values.iteritems():


кодируем признак города

In [603]:
encoder = ce.OneHotEncoder(cols=['city']) # указываем столбец для кодирования
type_bin = encoder.fit_transform(hotels['city'])
hotels = pd.concat([hotels, type_bin], axis=1)

# удаляем исходный признак национальности
hotels = hotels.drop(columns=['city'])

C:\Users\dolzh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\category_encoders\one_hot.py:145: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for cat_name, class_ in values.iteritems():


Работа с негативными отзывами

In [604]:
hotels['negative_review'] = hotels['negative_review'].apply(lambda x: x.strip().lower())
# список взят из (https://www.kaggle.com/code/motoborgrus/proj-3-hotels/notebook#%D0%9F%D1%80%D0%B8%D0%B7%D0%BD%D0%B0%D0%BA-hotel_address)
no_negative_review = ['No Negative',
 'Nothing',
 'nothing',
 'None',
 'N A',
 '',
 'Nothing really',
 'N a',
 'All good',
 'No complaints',
 'Nothing at all',
 'Nothing to dislike',
 'none',
 'Nil',
 'Everything was perfect',
 'Can t think of anything',
 'n a',
 'Absolutely nothing',
 'Everything was great',
 'Nothing to complain about',
 'Nothing not to like',
 'NA',
 'I liked everything',
 'NOTHING',
 'No',
 'Everything was good',
 'Liked everything',
 'Not much',
 'Nothing all good',
 'Nothing in particular',
 'Everything was fine',
 'Na',
 'All was good',
 'Nothing it was perfect',
 'Non',
 'Nothing comes to mind',
 'There was nothing I didn t like',
 'No complaints at all',
 'Nothing everything was perfect',
 'We liked everything',
 'Nothing I didn t like',
 'No negatives',
 'There was nothing to dislike',
 'Nothing everything was great',
 'nothing really',
 'Nothing to report',
 'Not a thing',
 'It was all good',
 'There was nothing we didn t like',
 'Loved everything',
 'No thing',
 'no complaints',
 'All ok',
 'all good',
 'There was nothing not to like',
 'No issues',
 'Nothing I can think of',
 'No bad experience',
 'Nothing to say',
 'No dislikes',
 'Nothing to mention',
 'no',
 'Everything was excellent',
 'Nothing we didn t like',
 'No problems',
 'Nothing to complain',
 'Nada',
 'Nothing bad to say',
 'everything',
 'nil',
 'nothing at all',
 'I loved everything',
 'Nothing bad',
 'All great']

no_negative_review = [i.lower() for i in no_negative_review]
hotels['is_negative'] = hotels['negative_review'].apply(lambda x: 0 if x in no_negative_review else 1)
hotels = hotels.drop(columns=['negative_review'])

Работа с позитивными отзывами

In [605]:
hotels['positive_review'] = hotels['positive_review'].apply(lambda x: x.strip().lower())
no_positive_review = ['no positive', 'nothing', 'n a', 'none', '', ' ' , 'not much', 'nothing at all']
hotels['is_positive'] = hotels['positive_review'].apply(lambda x: 0 if x in no_positive_review else 1)
hotels = hotels.drop(columns=['positive_review'])

Перевод признака даты в количество дней

In [606]:
hotels['days_from_rewiew'] = (hotels['review_date'].max() - hotels['review_date']).dt.days
# удаляем признак даты и признак days since rewiew, т.к. дублируется
hotels = hotels.drop(columns=['review_date'])
hotels = hotels.drop(columns=['days_since_review'])

Пропуски только в столбцах с широтой и долготой. Если в строке пропуск, удаляем строку

In [607]:
hotels = hotels.dropna(how='any', axis=0)

In [608]:
hotels.shape

(384355, 41)

In [609]:
hotels = hotels.drop_duplicates()

In [610]:
hotels.shape

(384044, 41)

Попробовать создать новые столбцы с данными из существующих данных или с использованием внешних источников.

## <center> 3. Создание новых признаков (из признака tags)

In [611]:
# функция перевода текста тегов в список
def tags_to_list(text):
    # list_len = len(text[2:-2].split('\', \''))
    tag_list = text[2:-2].split('\', \'')
    tag_list = [i.strip().lower() for i in tag_list]
    return tag_list

hotels['tags'] = hotels['tags'].apply(tags_to_list)

In [612]:
tags = []
for tag_list in hotels['tags']:
    for tag in tag_list:
        tags.append(tag)
        
from collections import Counter
cnt = Counter(tags)
cnt.most_common()

[('leisure trip', 311366),
 ('submitted from a mobile device', 229317),
 ('couple', 187915),
 ('stayed 1 night', 144740),
 ('stayed 2 nights', 99569),
 ('solo traveler', 80637),
 ('stayed 3 nights', 71312),
 ('business trip', 61548),
 ('group', 48709),
 ('family with young children', 45497),
 ('stayed 4 nights', 35333),
 ('double room', 26340),
 ('standard double room', 23822),
 ('superior double room', 23316),
 ('family with older children', 19669),
 ('deluxe double room', 18598),
 ('double or twin room', 16619),
 ('stayed 5 nights', 15436),
 ('standard double or twin room', 13058),
 ('classic double room', 12604),
 ('superior double or twin room', 9795),
 ('2 rooms', 9217),
 ('stayed 6 nights', 7338),
 ('standard twin room', 7233),
 ('single room', 7161),
 ('twin room', 6277),
 ('stayed 7 nights', 5496),
 ('executive double room', 4737),
 ('classic double or twin room', 4569),
 ('superior twin room', 4530),
 ('club double room', 4485),
 ('deluxe double or twin room', 4399),
 ('queen 

Создаем в датасете признаки, соответвующие наиболее часто встречающимся тегам

In [613]:
# for i in cnt.most_common():
#     if i[1] > 500 and 'night' not in i[0]:
#         feature = i[0]
#         hotels[feature] = hotels['tags'].apply(lambda x: 1 if feature in x else 0)

Создание признаков

In [614]:

hotels['is_leisure_trip'] = hotels['tags'].apply(lambda x: 1 if 'leisure trip' in x else 0)

hotels['is_mobile'] = hotels['tags'].apply(lambda x: 1 if 'submitted from a mobile device' in x else 0)

hotels['is_couple'] = hotels['tags'].apply(lambda x: 1 if 'couple' in x else 0)

hotels['is_solo'] = hotels['tags'].apply(lambda x: 1 if 'solo traveler' in x else 0)

hotels['is_business'] = hotels['tags'].apply(lambda x: 1 if 'business' in x else 0)

hotels['is_group_trip'] = hotels['tags'].apply(lambda x: 1 if 'group' in x else 0)

hotels['is_group_trip'] = hotels['tags'].apply(lambda x: 1 if 'group' in x else 0)

hotels['is_with_young_ch'] = hotels['tags'].apply(lambda x: 1 if 'young children' in x else 0)

hotels['is_with_older_ch'] = hotels['tags'].apply(lambda x: 1 if 'older children' in x else 0)

hotels['is_double'] = hotels['tags'].apply(lambda x: 1 if ('double room' in x) | ('2 rooms' in x) | ('twin' in x) else 0)

hotels['is_single'] = hotels['tags'].apply(lambda x: 1 if 'single' in x  else 0)


hotels['is_standard'] = hotels['tags'].apply(lambda x: 1 if 'standard' in x else 0)
hotels['is_classic'] = hotels['tags'].apply(lambda x: 1 if 'classic' in x else 0)
hotels['is_superior'] = hotels['tags'].apply(lambda x: 1 if 'superior' in x else 0)
hotels['is_deluxe'] = hotels['tags'].apply(lambda x: 1 if 'deluxe' in x else 0)
hotels['is_executive'] = hotels['tags'].apply(lambda x: 1 if 'executive' in x else 0)
hotels['is_club'] = hotels['tags'].apply(lambda x: 1 if 'club' in x else 0)
hotels['is_queen'] = hotels['tags'].apply(lambda x: 1 if 'queen' in x else 0)
hotels['is_king'] = hotels['tags'].apply(lambda x: 1 if 'king' in x else 0)
hotels['is_junior'] = hotels['tags'].apply(lambda x: 1 if 'king' in x else 0)

 

In [615]:
# определение количества ночей
# определяем из списка тегов тег с количеством ночей
def nights_in_hotel(tags_list):
    
    for tag in tags_list:
        if 'night' in tag:
            for i in tag.split():
                if i.isdigit():
                    return i
    return np.NaN
# применяем функцию, получаем признак в строковом формате, внутри которого число (присутствуют пропуски)
hotels['nights_in_hotel'] = hotels['tags'].apply(nights_in_hotel)
# заполняем пропуски модой
hotels['nights_in_hotel'] = hotels['nights_in_hotel'].fillna(hotels['nights_in_hotel'].mode()[0])
# приводим к типу int
hotels['nights_in_hotel'] = hotels['nights_in_hotel'].astype('int')

In [616]:
# удаляем признак тега
hotels = hotels.drop(columns=['tags'])

## <center> 4. Преобразование признаков

Для нормализации признаков применим RobustScaler т.к не особо уделял вниманиче очистке от выбросов и нет уверенности, что признаки распределены нормально

In [617]:
from sklearn import preprocessing

Применим различные преобразования над признаками - нормализацию


Выберем признаки, которые нормализируем

In [618]:
hotels = hotels.reset_index()

In [619]:
norm_cols_df = hotels[['additional_number_of_scoring', 'average_score', 'review_total_negative_word_counts', 'total_number_of_reviews', 'review_total_positive_word_counts', 'total_number_of_reviews_reviewer_has_given', 'lat', 'lng', 'nights_in_hotel']]

In [620]:
# Копируем названия столбцов, которые теряются при использовании fit_transform()
col_names = list(norm_cols_df.columns)

# инициализируем нормализатор RobustScaler
r_scaler = preprocessing.RobustScaler()

# копируем исходный датасет
df_r = r_scaler.fit_transform(norm_cols_df)

df_r = pd.DataFrame(df_r, columns=col_names)

Удаляем из датафрейма hotels признаки, до нормализации и добавляем нормализированные признаки

In [621]:
hotels = hotels.drop(columns=['additional_number_of_scoring', 'average_score', 'review_total_negative_word_counts', 'total_number_of_reviews', 'review_total_positive_word_counts', 'total_number_of_reviews_reviewer_has_given', 'lat', 'lng', 'nights_in_hotel'])

hotels = pd.concat([hotels, df_r], axis=1)

## <center> 5. Отбор признаков

Используем анализ мультиколлинеарности как шаг отбора признаков для модели.

In [622]:
hotels_corr = hotels.corr()

for row in hotels_corr.index:
    for col in hotels_corr.columns:
        if hotels_corr.loc[row, col] != 1 and abs(hotels_corr.loc[row, col]) > 0.65:
            print(row, col)

city_1 lng
city_5 lng
city_6 lat
additional_number_of_scoring total_number_of_reviews
total_number_of_reviews additional_number_of_scoring
lat city_6
lng city_1
lng city_5


В результате анализа мультиколинеарности, необходимо удалить признаки


In [623]:
hotels = hotels.drop(columns=['additional_number_of_scoring', 'city_6', 'city_1', 'city_5'])

## <center> Дальнейший расчет

In [624]:
# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели  
# Х - данные с информацией об отелях, у - целевая переменная (рейтинги отелей)  
y = hotels['reviewer_score'] 
X = hotels.drop(['reviewer_score'], axis = 1)  


In [625]:
# Загружаем специальный инструмент для разбивки:  
from sklearn.model_selection import train_test_split  

In [626]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.  
# Для тестирования мы будем использовать 25% от исходного датасета.  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [627]:
# Импортируем необходимые библиотеки:  
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели  
from sklearn import metrics # инструменты для оценки точности модели  
  
# Создаём модель  
regr = RandomForestRegressor(n_estimators=100)  
      
# Обучаем модель на тестовом наборе данных  
regr.fit(X_train, y_train)  
      
# Используем обученную модель для предсказания рейтинга отелей в тестовой выборке.  
# Предсказанные значения записываем в переменную y_pred  
y_pred = regr.predict(X_test)  


In [628]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они отличаются  
# Метрика называется Mean Absolute Percentage Error (MAPE) и показывает среднюю абсолютную процентную ошибку предсказанных значений от фактических.  
print('MAPE:', metrics.mean_absolute_percentage_error(y_test, y_pred))

MAPE: 0.13339875420488506


Небольшой бонус:


In [629]:
# # убираем признаки которые еще не успели обработать, 
# # модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
# object_columns = [s for s in hotels.columns if hotels[s].dtypes == 'object']
# hotels.drop(object_columns, axis = 1, inplace=True)

# # заполняем пропуски самым простым способом
# hotels = hotels.fillna(0)